In [1]:
import os
import sys
import pandas as pd
import re
from sklearn.svm import LinearSVC
from sklearn.externals import joblib

/home/sravani/.local/lib/python2.7/site-packages/sklearn/utils/fixes.py:313: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  _nan_object_mask = _nan_object_array != _nan_object_array


In [2]:
train_path = '/media/sdi/sravani/Jigsaws_stip/features2/video_features/train'
test_path = '/media/sdi/sravani/Jigsaws_stip/features2/video_features/test'

train_write_path = '/media/sdi/sravani/Jigsaws_stip/features2/scores/train'
test_write_path = '/media/sdi/sravani/Jigsaws_stip/features2/scores/test'

In [3]:
action_folders_train = os.listdir(train_path) 
action_folders_test = os.listdir(test_path) 

action_folders_train.sort()
action_folders_test.sort()
#action_folders_train
#action_folders_test
len(action_folders_train)

3

In [4]:
action_num_features = {}
df = pd.DataFrame()

In [5]:
for one_action_folder in action_folders_train:
    wd = train_path+'/'+one_action_folder+'/'
    one_action_files = [f for f in os.listdir(wd) if not re.findall('scores',f)]
    print(one_action_files)
    df1 = pd.DataFrame()
    for files in one_action_files:
        temp_df = pd.read_csv(wd+files,header = None)
        #print(temp_df.shape)
        df1 = pd.concat([df1,temp_df])
    df = pd.concat([df,df1])
    print(df1.shape)
    action_num_features[one_action_folder] = df1.shape[0]

['knot_stip_48_feat.csv', 'knot_stip_43_feat.csv', 'knot_stip_7_feat.csv', 'knot_stip_65_feat.csv', 'knot_stip_60_feat.csv', 'knot_stip_15_feat.csv', 'knot_stip_58_feat.csv', 'knot_stip_12_feat.csv', 'knot_stip_50_feat.csv', 'knot_stip_71_feat.csv', 'knot_stip_54_feat.csv', 'knot_stip_49_feat.csv', 'knot_stip_44_feat.csv', 'knot_stip_36_feat.csv', 'knot_stip_31_feat.csv', 'knot_stip_68_feat.csv', 'knot_stip_67_feat.csv', 'knot_stip_1_feat.csv', 'knot_stip_4_feat.csv', 'knot_stip_66_feat.csv', 'knot_stip_62_feat.csv', 'knot_stip_17_feat.csv', 'knot_stip_11_feat.csv', 'knot_stip_64_feat.csv', 'knot_stip_39_feat.csv', 'knot_stip_13_feat.csv', 'knot_stip_3_feat.csv', 'knot_stip_8_feat.csv', 'knot_stip_33_feat.csv', 'knot_stip_35_feat.csv', 'knot_stip_6_feat.csv', 'knot_stip_2_feat.csv', 'knot_stip_40_feat.csv', 'knot_stip_37_feat.csv', 'knot_stip_41_feat.csv', 'knot_stip_63_feat.csv', 'knot_stip_53_feat.csv', 'knot_stip_34_feat.csv', 'knot_stip_5_feat.csv', 'knot_stip_32_feat.csv', 'knot_s

In [6]:
all_actions = list(action_num_features.keys())
all_actions.sort()
print(all_actions)
num_feat = sum(action_num_features.values())

['Knot_Tying', 'Needle_Passing', 'Suturing']


In [7]:
df = df.drop(columns=[162,163,164])
print(df.shape)
features_list = df.values.tolist()

(7626394, 162)


In [8]:
print(len(features_list))
print(sum(action_num_features.values()))
all_actions

7626394
7626394


['Knot_Tying', 'Needle_Passing', 'Suturing']

In [9]:
start = 0
for action in all_actions:
    print(action, start)
    num_ones = action_num_features[action]
    labels = [-1]*num_feat
    for i in range(start,start+num_ones):
        labels[i] = 1
    clf = LinearSVC(class_weight='balanced',max_iter = 2000)
    clf.fit(features_list, labels)
    joblib.dump(clf,'JIGSAW_clf_%s.pkl'%(action))
    start += num_ones

('Knot_Tying', 0)


MemoryError: 

In [16]:
for i in all_actions:
    #df = pd.DataFrame()
    train_one_action_files = [ f for f in os.listdir(train_path+'/'+i) if not re.findall('scores',f)]
    print(train_one_action_files)
    clf = joblib.load('JIGSAW_clf_%s.pkl'%(i))
    #print(test_one_action_files)
    print(i)
    for j in train_one_action_files:
        df = pd.read_csv(train_path+'/'+i+'/'+j,header=None)
        #df = pd.concat([df1])
        print("--------------------------")
        print(train_path+'/'+i+'/'+j)
        df1 = df.drop(columns=[162,163,164])
        #print(df1.head())
        df1 = df1.values.tolist()
        
        scores= clf.decision_function(df1)
        scores = pd.DataFrame(scores,columns=[165])
        #df = pd.DataFrame(df)
        df = pd.concat([df,scores],axis=1)
        print(df.shape)
        df = df[df[165]>0]
        #print(df.shape)
        #print j
        #print("----------------")
        #print(j[len(i+'-stip'):-4])
        #df.to_csv(train_write_path+'/'+i+'/'+j[len(i+'-stip'):-4]+'.csv',header=False,index=False)
        df.to_csv(train_write_path+'/'+i+'/'+j[:-4]+'.csv',header=False,index=False)
        #print(df.head())
        print(df.shape)
        print("==")
    print("----")

['stip1_41_feat.csv', 'stip1_35_feat.csv', 'stip1_20_feat.csv', 'stip1_29_feat.csv', 'stip1_70_feat.csv', 'stip1_57_feat.csv', 'stip1_38_feat.csv', 'stip1_73_feat.csv', 'stip1_19_feat.csv', 'stip1_43_feat.csv', 'stip1_09_feat.csv', 'stip1_11_feat.csv', 'stip1_66_feat.csv', 'stip1_21_feat.csv', 'stip1_50_feat.csv', 'stip1_53_feat.csv', 'stip1_72_feat.csv', 'stip1_26_feat.csv', 'stip1_34_feat.csv', 'stip1_25_feat.csv', 'stip1_07_feat.csv', 'stip1_42_feat.csv', 'stip1_76_feat.csv', 'stip1_71_feat.csv', 'stip1_63_feat.csv', 'stip1_13_feat.csv', 'stip1_56_feat.csv', 'stip1_17_feat.csv', 'stip1_47_feat.csv', 'stip1_06_feat.csv', 'stip1_01_feat.csv', 'stip1_04_feat.csv', 'stip1_44_feat.csv', 'stip1_02_feat.csv', 'stip1_54_feat.csv', 'stip1_61_feat.csv', 'stip1_05_feat.csv', 'stip1_69_feat.csv', 'stip1_28_feat.csv', 'stip1_48_feat.csv', 'stip1_67_feat.csv', 'stip1_10_feat.csv', 'stip1_64_feat.csv', 'stip1_77_feat.csv', 'stip1_60_feat.csv', 'stip1_15_feat.csv', 'stip1_33_feat.csv', 'stip1_75_fe

(12267, 166)
==
--------------------------
/home/sravani/stip_code/data/stip_files/features1/video_features/train/stip1/stip1_61_feat.csv
(9491, 166)
stip1_61_feat.csv
----------------
eat
(7599, 166)
==
--------------------------
/home/sravani/stip_code/data/stip_files/features1/video_features/train/stip1/stip1_05_feat.csv
(20942, 166)
stip1_05_feat.csv
----------------
eat
(15968, 166)
==
--------------------------
/home/sravani/stip_code/data/stip_files/features1/video_features/train/stip1/stip1_69_feat.csv
(19886, 166)
stip1_69_feat.csv
----------------
eat
(14827, 166)
==
--------------------------
/home/sravani/stip_code/data/stip_files/features1/video_features/train/stip1/stip1_28_feat.csv
(12177, 166)
stip1_28_feat.csv
----------------
eat
(9532, 166)
==
--------------------------
/home/sravani/stip_code/data/stip_files/features1/video_features/train/stip1/stip1_48_feat.csv
(16634, 166)
stip1_48_feat.csv
----------------
eat
(13832, 166)
==
--------------------------
/home/srav

stip2
--------------------------
/home/sravani/stip_code/data/stip_files/features1/video_features/train/stip2/stip2_47_feat.csv
(12307, 166)
stip2_47_feat.csv
----------------
eat
(10622, 166)
==
--------------------------
/home/sravani/stip_code/data/stip_files/features1/video_features/train/stip2/stip2_13_feat.csv
(10369, 166)
stip2_13_feat.csv
----------------
eat
(7905, 166)
==
--------------------------
/home/sravani/stip_code/data/stip_files/features1/video_features/train/stip2/stip2_77_feat.csv
(12459, 166)
stip2_77_feat.csv
----------------
eat
(10266, 166)
==
--------------------------
/home/sravani/stip_code/data/stip_files/features1/video_features/train/stip2/stip2_68_feat.csv
(20805, 166)
stip2_68_feat.csv
----------------
eat
(15990, 166)
==
--------------------------
/home/sravani/stip_code/data/stip_files/features1/video_features/train/stip2/stip2_43_feat.csv
(9624, 166)
stip2_43_feat.csv
----------------
eat
(7131, 166)
==
--------------------------
/home/sravani/stip_c

(18484, 166)
stip2_36_feat.csv
----------------
eat
(12241, 166)
==
--------------------------
/home/sravani/stip_code/data/stip_files/features1/video_features/train/stip2/stip2_26_feat.csv
(12571, 166)
stip2_26_feat.csv
----------------
eat
(9934, 166)
==
--------------------------
/home/sravani/stip_code/data/stip_files/features1/video_features/train/stip2/stip2_65_feat.csv
(9660, 166)
stip2_65_feat.csv
----------------
eat
(8113, 166)
==
--------------------------
/home/sravani/stip_code/data/stip_files/features1/video_features/train/stip2/stip2_25_feat.csv
(18240, 166)
stip2_25_feat.csv
----------------
eat
(14419, 166)
==
--------------------------
/home/sravani/stip_code/data/stip_files/features1/video_features/train/stip2/stip2_60_feat.csv
(16583, 166)
stip2_60_feat.csv
----------------
eat
(13669, 166)
==
--------------------------
/home/sravani/stip_code/data/stip_files/features1/video_features/train/stip2/stip2_02_feat.csv
(14342, 166)
stip2_02_feat.csv
----------------
eat
(

--------------------------
/home/sravani/stip_code/data/stip_files/features1/video_features/train/stip3/stip3_08_feat.csv
(7882, 166)
stip3_08_feat.csv
----------------
eat
(5111, 166)
==
--------------------------
/home/sravani/stip_code/data/stip_files/features1/video_features/train/stip3/stip3_03_feat.csv
(14353, 166)
stip3_03_feat.csv
----------------
eat
(9824, 166)
==
--------------------------
/home/sravani/stip_code/data/stip_files/features1/video_features/train/stip3/stip3_41_feat.csv
(3710, 166)
stip3_41_feat.csv
----------------
eat
(2956, 166)
==
--------------------------
/home/sravani/stip_code/data/stip_files/features1/video_features/train/stip3/stip3_05_feat.csv
(12318, 166)
stip3_05_feat.csv
----------------
eat
(8888, 166)
==
--------------------------
/home/sravani/stip_code/data/stip_files/features1/video_features/train/stip3/stip3_01_feat.csv
(1359, 166)
stip3_01_feat.csv
----------------
eat
(857, 166)
==
--------------------------
/home/sravani/stip_code/data/sti

(5464, 166)
==
--------------------------
/home/sravani/stip_code/data/stip_files/features1/video_features/train/stip3/stip3_20_feat.csv
(4806, 166)
stip3_20_feat.csv
----------------
eat
(2597, 166)
==
--------------------------
/home/sravani/stip_code/data/stip_files/features1/video_features/train/stip3/stip3_29_feat.csv
(5399, 166)
stip3_29_feat.csv
----------------
eat
(3534, 166)
==
--------------------------
/home/sravani/stip_code/data/stip_files/features1/video_features/train/stip3/stip3_65_feat.csv
(4661, 166)
stip3_65_feat.csv
----------------
eat
(3773, 166)
==
--------------------------
/home/sravani/stip_code/data/stip_files/features1/video_features/train/stip3/stip3_51_feat.csv
(4058, 166)
stip3_51_feat.csv
----------------
eat
(3316, 166)
==
--------------------------
/home/sravani/stip_code/data/stip_files/features1/video_features/train/stip3/stip3_43_feat.csv
(4546, 166)
stip3_43_feat.csv
----------------
eat
(3343, 166)
==
--------------------------
/home/sravani/stip

(7400, 166)
==
--------------------------
/home/sravani/stip_code/data/stip_files/features1/video_features/train/stip4/stip4_03_feat.csv
(10589, 166)
stip4_03_feat.csv
----------------
eat
(7139, 166)
==
--------------------------
/home/sravani/stip_code/data/stip_files/features1/video_features/train/stip4/stip4_66_feat.csv
(8163, 166)
stip4_66_feat.csv
----------------
eat
(5727, 166)
==
--------------------------
/home/sravani/stip_code/data/stip_files/features1/video_features/train/stip4/stip4_46_feat.csv
(6288, 166)
stip4_46_feat.csv
----------------
eat
(4330, 166)
==
--------------------------
/home/sravani/stip_code/data/stip_files/features1/video_features/train/stip4/stip4_74_feat.csv
(8584, 166)
stip4_74_feat.csv
----------------
eat
(5134, 166)
==
--------------------------
/home/sravani/stip_code/data/stip_files/features1/video_features/train/stip4/stip4_73_feat.csv
(3900, 166)
stip4_73_feat.csv
----------------
eat
(3004, 166)
==
--------------------------
/home/sravani/sti

--------------------------
/home/sravani/stip_code/data/stip_files/features1/video_features/train/stip4/stip4_59_feat.csv
(1925, 166)
stip4_59_feat.csv
----------------
eat
(1810, 166)
==
--------------------------
/home/sravani/stip_code/data/stip_files/features1/video_features/train/stip4/stip4_19_feat.csv
(8925, 166)
stip4_19_feat.csv
----------------
eat
(6238, 166)
==
--------------------------
/home/sravani/stip_code/data/stip_files/features1/video_features/train/stip4/stip4_40_feat.csv
(8895, 166)
stip4_40_feat.csv
----------------
eat
(4891, 166)
==
--------------------------
/home/sravani/stip_code/data/stip_files/features1/video_features/train/stip4/stip4_67_feat.csv
(10033, 166)
stip4_67_feat.csv
----------------
eat
(4378, 166)
==
--------------------------
/home/sravani/stip_code/data/stip_files/features1/video_features/train/stip4/stip4_62_feat.csv
(13272, 166)
stip4_62_feat.csv
----------------
eat
(7005, 166)
==
--------------------------
/home/sravani/stip_code/data/st

(11075, 166)
==
--------------------------
/home/sravani/stip_code/data/stip_files/features1/video_features/train/stip5/stip5_38_feat.csv
(8704, 166)
stip5_38_feat.csv
----------------
eat
(5408, 166)
==
--------------------------
/home/sravani/stip_code/data/stip_files/features1/video_features/train/stip5/stip5_13_feat.csv
(17172, 166)
stip5_13_feat.csv
----------------
eat
(12388, 166)
==
--------------------------
/home/sravani/stip_code/data/stip_files/features1/video_features/train/stip5/stip5_45_feat.csv
(18742, 166)
stip5_45_feat.csv
----------------
eat
(14046, 166)
==
--------------------------
/home/sravani/stip_code/data/stip_files/features1/video_features/train/stip5/stip5_10_feat.csv
(10557, 166)
stip5_10_feat.csv
----------------
eat
(7624, 166)
==
--------------------------
/home/sravani/stip_code/data/stip_files/features1/video_features/train/stip5/stip5_52_feat.csv
(6508, 166)
stip5_52_feat.csv
----------------
eat
(4702, 166)
==
--------------------------
/home/sravan

(5838, 166)
==
--------------------------
/home/sravani/stip_code/data/stip_files/features1/video_features/train/stip5/stip5_16_feat.csv
(2928, 166)
stip5_16_feat.csv
----------------
eat
(2348, 166)
==
--------------------------
/home/sravani/stip_code/data/stip_files/features1/video_features/train/stip5/stip5_28_feat.csv
(14191, 166)
stip5_28_feat.csv
----------------
eat
(10693, 166)
==
--------------------------
/home/sravani/stip_code/data/stip_files/features1/video_features/train/stip5/stip5_27_feat.csv
(19381, 166)
stip5_27_feat.csv
----------------
eat
(12743, 166)
==
--------------------------
/home/sravani/stip_code/data/stip_files/features1/video_features/train/stip5/stip5_41_feat.csv
(11370, 166)
stip5_41_feat.csv
----------------
eat
(7818, 166)
==
--------------------------
/home/sravani/stip_code/data/stip_files/features1/video_features/train/stip5/stip5_74_feat.csv
(18331, 166)
stip5_74_feat.csv
----------------
eat
(11360, 166)
==
--------------------------
/home/srava

In [7]:
for i in all_actions:
    #df = pd.DataFrame()
    test_one_action_files = [ f for f in os.listdir(test_path+'/'+i) if not re.findall('scores',f)]
    print(test_one_action_files)
    clf = joblib.load('JIGSAW_clf_%s.pkl'%(i))
    #print(test_one_action_files)
    print(i)
    for j in test_one_action_files:
        df = pd.read_csv(test_path+'/'+i+'/'+j,header=None)
        #df = pd.concat([df1])
        df1 = df.drop(columns=[162,163,164])
        #print(df1.head())
        df1 = df1.values.tolist()
        
        scores= clf.decision_function(df1)
        scores = pd.DataFrame(scores,columns=[165])
        #df = pd.DataFrame(df)
        df = pd.concat([df,scores],axis=1)
        print(df.shape)
        df = df[df[165]>0]
        #print(df.shape)
        df.to_csv(test_write_path+'/'+i+'/'+j[:-4]+'.csv',header=False,index=False)
        #print(df.head())
        print(df.shape)
        print("==")
    print("----")

['stip1_95_feat.csv', 'stip1_90_feat.csv', 'stip1_85_feat.csv', 'stip1_79_feat.csv', 'stip1_81_feat.csv', 'stip1_103_feat.csv', 'stip1_82_feat.csv', 'stip1_91_feat.csv', 'stip1_83_feat.csv', 'stip1_87_feat.csv', 'stip1_80_feat.csv', 'stip1_100_feat.csv', 'stip1_104_feat.csv', 'stip1_96_feat.csv', 'stip1_94_feat.csv', 'stip1_101_feat.csv', 'stip1_98_feat.csv', 'stip1_88_feat.csv', 'stip1_97_feat.csv', 'stip1_105_feat.csv', 'stip1_89_feat.csv', 'stip1_102_feat.csv', 'stip1_99_feat.csv', 'stip1_106_feat.csv', 'stip1_84_feat.csv', 'stip1_86_feat.csv', 'stip1_92_feat.csv', 'stip1_93_feat.csv']
stip1
(15710, 166)
(11323, 166)
==
(15534, 166)
(10639, 166)
==
(8252, 166)
(6480, 166)
==
(9118, 166)
(5957, 166)
==
(21142, 166)
(14638, 166)
==
(19166, 166)
(14678, 166)
==
(14073, 166)
(11631, 166)
==
(16272, 166)
(11736, 166)
==
(11401, 166)
(8554, 166)
==
(8766, 166)
(6092, 166)
==
(14709, 166)
(10901, 166)
==
(19513, 166)
(13290, 166)
==
(14930, 166)
(10462, 166)
==
(18624, 166)
(12879, 166)
==